In [1]:
from false_positives_negatives import *
%matplotlib widget

# H. Pylori
*Helicobacter Pylori* is a type of bacteria that can cause infections in the stomach. Symptoms of *H. pylori* include painful stomach ulcers. A common test for the infection is the urea breath test. After the patient ingests a tablet or drink containing urea enriched with the isotope carbon-13 (rather than the usual carbon-12), the test measures the proportion of carbon-13 in the carbon dioxide in the collected breath sample. If the amount of carbon-13 is in excess of the normal amount of carbon-13 in one's breath, then it suggests the presence of the bacteria in the stomach, because the bacteria have converted the ingested urea into carbon dioxide. The test result is how much excess carbon-13 is found in one's breath. For example, a test result of 5 means that there is 5 times as much carbon-13 as usual.

As study was done to find the distribution of the urea test results of subjects with or without *H. pylori* infection. Below is a histogram that shows the number of patients with a particular test result. Patients without the infection are plotted in blue; those who are infected are plotted in orange.

https://onlinelibrary.wiley.com/doi/epdf/10.1046/j.1365-2036.2003.01417.x

In [4]:
config_hpylori = {
    'title': 'H. pylori breath test',
    'mean_neg': 0.16,
    'mean_pos': 30.2,
    'std_neg': 0.36,
    'std_pos': 2.5,
    'p_pos': .549,
    'label_neg': 'No H. pylori infection',
    'label_pos': 'H. pylori infected'
}
test_hpylori = Test(config_hpylori)
test_hpylori.generate(1000)
test_hpylori.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=18.047805386413003, description='t', max=38.8974157361912, min=-2.8018…

Try slowly moving the slider from side to side. This sets the "threshold" value that delineates positive and negative diagnoses. That is, if a patient's urea test result is above this value, they will be told they are "positive"; if the patient's result is below this value, they will be told they are "negative". As you move the slider, pay attention to the number of true/false positives/negatives.

You are the designer of this test. Your job is to decide on a value for this threshold, which will be used by all the clinical laboratories to make diagnoses. **What threshold value do you choose?**

Answer here

---
In most situations, the delineation between positives and negatives are not that clear. In other words, the test results of positives may often over lap with those of negatives. Let us see such a case.

# Pregnancy Test

In the first days of pregnancy, the body produces a hormone called human chorionic gonadotropin (hCG). A home pregnancy kit works by detecting the presence of this hormone in the urine as an indication of pregnancy. The concentration of hCG is measured in units of milli-international units per millilitre (mIU/mL). hCG levels in a non-pregnant woman is typically less than 5 mIU/mL, and the number increases rapidly after the implantation of the embryo.

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4119102/

http://www.clinlabnavigator.com/human-chorionic-gonadotropin-pregnancy-test.html

In [5]:
config_preg_day9 = {
    'title': 'hCG Levels in Urine on 9-th Day of Pregnancy',
    'mean_neg': 0.1,
    'mean_pos': 4.04,
    'std_neg': 0.02,
    'std_pos': 1.9,
    'floor': 0,
    'p_pos': 0.8,
    'label_neg': 'Not pregnant',
    'label_pos': 'Pregnant'
}
test_preg_day9 = Test(config_preg_day9)
test_preg_day9.generate(1000)
test_preg_day9.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=5.141573541605803, description='t', max=10.797304437372189, min=-0.514…

Home pregnancy test kits can only detect hCG if it is above a certain level. This is called the "sensitivity". If the test returns positive for all results above the sensitivity level and negative otherwise, **what should the sensitivity level of a home pregnancy test be in order to ensure (1) no false positives and (2) less than 10% false negatives on the 9-th day of pregnancy?** Try using the slider again to adjust the sensitivity/threshold and observe the changing numbers below the graph.

In [6]:
config_preg_day11 = {
    'title': 'hCG Levels in Urine on 9-th Day of Pregnancy',
    'mean_neg': 0.1,
    'mean_pos': 25.04,
    'std_neg': 0.02,
    'std_pos': 8.1,
    'floor': 0,
    'p_pos': 0.8,
    'label_neg': 'Not pregnant',
    'label_pos': 'Pregnant'
}
test_preg_day11 = Test(config_preg_day11)
test_preg_day11.generate(1000)
test_preg_day11.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=26.3995697418427, description='t', max=55.43909645786967, min=-2.63995…

__What should the sensitivity level of a home pregnancy test be in order to ensure (1) no false positives and (2) less than 10% false negatives on the 11-th day of pregnancy?__

# Smoke detector

Smoke detectors work by measuring how much the air is dispersing light. The more smoke there is, the less light will be able to pass through the air. The quantity measured by a smoke detector is called the "optical density", which has the units of "percent per metre" (%/m). The goal of a smoke detector is to warn inhabitants of a house of a possible dangerous fire.

The following graph shows the number of times a certain value of optical density has been measured in various scenarios. The blue histogram counts those times when there is no dangerous fire (perhaps there is some greasy haze in the room from a stirfry), and the orage histogram counts those times when there is a dangerous fire (perhaps a stack of paper has been lit up by a cigarette).

https://iafss.org/publications/aofst/3/319/view/aofst_3-319.pdf

In [7]:
config_CO_detector = {
    'title': 'Optical Density (%/m)',
    'mean_neg': 0.2,
    'mean_pos': 3,
    'std_neg': 0.1,
    'std_pos': 1.5,
    'floor': 0,
    'p_pos': 0.6,
    'label_neg': 'No Fire',
    'label_pos': 'Fire'
}
test_CO_detector = Test(config_CO_detector)
test_CO_detector.generate(1000)
test_CO_detector.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=4.094865733585329, description='t', max=8.599218040529193, min=-0.4094…

Suppose you are the designer of a smoke detector, tasked to set a "sensitivity" or threshold value of optical density, such that the smoke detector will sound its alarm when the optical density in the room exceeds that value. But first, let's consider a few points.

__In the context of the smoke detector's alarm, what scenarios would be considered a true positive, a false positive, a true negative, or a false negative?__

__For each of these scenarios, what is the consequence to the inhabitants of the house?__

__With reference to these consequences, what criteria would you be considering when setting a threshold value for a smoke detector?__

Drag the slider in the graph above to change the threshold value, while paying attention to the false positive and false negative rates. __Choose a value that best satisfies the criteria you set forth above. Explain whether you are satisfied with your choice.__

You have no doubt experienced an annoying smoke detector that has gone off, but nothing has burnt down, prompting a pointless trip by the fire brigade. __Why do situations like this keep happening?__

# SAT scores vs admission

The binary classification of positives and negatives as well as the trade-off between false positive rate and false negative rate have important social consequences, especially when the subjects of classification are individual humans.

One example of such a classification is the admission of undergraduates to universities on the basis of standardised test performance. A good score on tests like the ACT and SAT is known to correlate strongly with a high GPA in university. As the following graph shows, the higher the ACT score, the more likely it is that a student with that ACT score will achieve a high GPA.

<img src="ACT_GPA.jpg" width="600px">

Suppose you are a college admissions officer who is given the task of setting an ACT score cut-off, so that only students who score above this cut-off will be considered for admission at all. The college administration tells you that whatever value you choose, they would like at least 50% of the admitted students to eventually have a GPA of over 3.0 in their first year, in order to maintain the university's historically positive reputation.

https://www.act.org/content/dam/act/unsecured/documents/R1645-act-composite-to-fygpa-2017-05.pdf

__With reference to the above graph, is there a correlation between ACT score and academic performance in the first year? Would you say that ACT score is a good admissions criterion?__

__In the context of admission, what is considered a "positive diagnosis" or a "negative diagnosis"?__

__With reference to the administration's demands, what scenarios would be considered a true positive, a false positive, a true negative, or a false negative?__

__For each of thse scenarios, what are its consequences?__

In [2]:
config = {
    'title': 'ACT Scores',
    'label_neg': 'First year GPA <3.0',
    'label_pos': 'First year GPA ≥3.0'
}
act_data = np.genfromtxt('ACT_GPA.csv', delimiter=',', skip_header=1)
# ACT score, N, % above 3.0, N above 3.0, % above 3.5, N above 3.5
data = np.vstack((act_data[:,0], act_data[:,3], act_data[:,1]-act_data[:,3])).T
test_act = Test(config=config, data=data)
test_act.generate(10000)
test_act.plot(stepsize=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=23.0, description='t', max=41.0, min=6.0, step=1.0), Output()), _dom_c…

__What ACT score would you set as the cut-off for admission? Explain your reasoning for why it isn't higher or lower than what you've chosen.__

Suppose all those who scored lower than your cut-off are outright rejected. __Considering individual students, why would such an admissions rule be problematic?__

__With reference to the above histogram and the false positives and false negative rates, would you say that ACT score is a good admissions criterion?__

# Spam vs ham or elicit content censorship (add question on realist vs conventionalist definition)

* implement different scenarios with different distributions
* base rate neglect by varying p_pos and asking how likely they are to have the disease given a positive diagnosis